In [ ]:

# # Deep Causal Models with EconML: DRLearner and DeepIV (Colab Version)

# 📦 Install dependencies
!pip install econml xgboost scikit-learn pandas matplotlib seaborn torch

# 📁 Upload your data: peacock_user_data_with_renewed_and_propensity.csv
from google.colab import files
uploaded = files.upload()

# 📊 Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from econml.dr import DRLearner
from econml.iv.nnet import DeepIVEstimator
from econml.utilities import hstack

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# 📥 Load data
df = pd.read_csv("peacock_user_data_with_renewed_and_propensity.csv")

# Feature setup
X = df.drop(columns=["user_id", "assigned_promo", "renewed", "propensity_score"])
T = df["assigned_promo"]
Y = df["renewed"]

# Known CATE for evaluation
tau_x = (
    0.4
    - 0.7 * df["prior_engagement_score"]
    + 0.1 * (df["device_type"] == "roku").astype(int)
    + 0.05 * (df["has_kids_profile"] == 1).astype(int)
)

# Train/test split
X_train, X_test, T_train, T_test, Y_train, Y_test, tau_train, tau_test = train_test_split(
    X, T, Y, tau_x, test_size=0.3, random_state=42
)

# Column Transformer
numeric_features = ["tenure_months", "prior_engagement_score", "weekly_watch_hours", "num_devices"]
categorical_features = ["device_type", "payment_method", "account_type", "region", "has_kids_profile", "promo_eligible"]

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
])

X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

# 🔧 DRLearner with neural net models
from sklearn.ensemble import GradientBoostingRegressor
from econml.metalearners import TLearner
from econml.models import KerasModel

import tensorflow as tf
from tensorflow.keras import layers, models

def build_keras_model(input_shape):
    model = models.Sequential([
        layers.Input(shape=(input_shape,)),
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")
    return model

# DRLearner
dr_learner = DRLearner(
    model_propensity=GradientBoostingRegressor(),
    model_regression=GradientBoostingRegressor(),
    model_final=KerasModel(model_builder=lambda: build_keras_model(X_train_proc.shape[1]),
                           fit_kwargs={'epochs': 30, 'verbose': 0})
)

dr_learner.fit(Y_train, T_train, X=X_train_proc)
cate_dr = dr_learner.effect(X_test_proc)

# PEHE evaluation
from sklearn.metrics import mean_squared_error
pehe_dr = np.sqrt(mean_squared_error(tau_test, cate_dr))

# Plot
plt.figure(figsize=(6, 4))
sns.histplot(cate_dr, kde=True, bins=30)
plt.title(f"DRLearner (PEHE: {pehe_dr:.3f})")
plt.xlabel("Estimated CATE")
plt.grid(True)
plt.tight_layout()
plt.show()

# 🚀 Optional: DeepIV if IV is available (instrument z ≠ T)
# Simulate an instrument for demo
Z = df["promo_eligible"]
Z_train, Z_test = train_test_split(Z, test_size=0.3, random_state=42)

# DeepIV requires outcome, treatment, instrument, and covariates
deepiv = DeepIVEstimator(
    n_components=10,
    m=lambda z, x: build_keras_model(x.shape[1]),
    h=lambda t, x: build_keras_model(x.shape[1]),
    n_samples=1,
    optimizer="adam",
    loss="mse",
    first_stage_options={'epochs': 30, 'verbose': 0},
    second_stage_options={'epochs': 30, 'verbose': 0}
)

# DeepIV fit
deepiv.fit(Y_train, T_train, Z_train, X=X_train_proc)

# Estimate CATE at T=1 vs T=0
cate_deepiv = deepiv.effect(X_test_proc, T0=0, T1=1)
pehe_deepiv = np.sqrt(mean_squared_error(tau_test, cate_deepiv))

# Plot DeepIV
plt.figure(figsize=(6, 4))
sns.histplot(cate_deepiv, kde=True, bins=30)
plt.title(f"DeepIV (PEHE: {pehe_deepiv:.3f})")
plt.xlabel("Estimated CATE")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

# 📈 Policy Value Evaluation and Uplift-Based Targeting

# Sort test users by predicted uplift (CATE)
test_df = pd.DataFrame({
    "user_id": X_test.index,
    "cate": cate_preds,
    "true_tau": tau_test,
    "treatment": T_test,
    "outcome": Y_test
}).reset_index(drop=True)

# Rank by predicted CATE
test_df_sorted = test_df.sort_values(by="cate", ascending=False).reset_index(drop=True)

# Define budget: top 20% of users
budget_fraction = 0.2
n_target = int(budget_fraction * len(test_df_sorted))

# Evaluate policy value
def policy_value(df, treat_col, outcome_col):
    return df.loc[df[treat_col] == 1, outcome_col].mean()

# Scenarios
treat_all = test_df.copy()
treat_all["treatment"] = 1
value_all = policy_value(treat_all, "treatment", "outcome")

treat_none = test_df.copy()
treat_none["treatment"] = 0
value_none = policy_value(treat_none, "treatment", "outcome")

uplift_target = test_df.copy()
uplift_target["treatment"] = 0
uplift_target.loc[test_df_sorted.head(n_target).index, "treatment"] = 1
value_targeted = policy_value(uplift_target, "treatment", "outcome")

# 📊 Print comparison
print("Policy Value Comparison:")
print(f"Treat All:    {value_all:.4f}")
print(f"Treat None:   {value_none:.4f}")
print(f"Uplift-Based: {value_targeted:.4f} (Top {budget_fraction*100:.0f}%)")

# Plot top uplift users
plt.figure(figsize=(10, 4))
sns.histplot(test_df_sorted["cate"].head(n_target), bins=30, kde=True, color="green")
plt.axvline(0, color="red", linestyle="--")
plt.title("Top Uplift Scores (Targeted Segment)")
plt.xlabel("Predicted Uplift (CATE)")
plt.tight_layout()
plt.show()


In [ ]:

# 🎯 Targeting Precision Evaluation

# Define actual uplift = observed Y1 - Y0 for each user
# Here we approximate it using true uplift tau and simulate based on prediction rank

test_df_sorted["true_positive"] = test_df_sorted["true_tau"] > 0
test_df_sorted["predicted_positive"] = test_df_sorted["cate"] > 0

# Precision at top-k (top 20%)
top_k_df = test_df_sorted.head(n_target)

precision_top_k = top_k_df["true_positive"].mean()
overall_positive_rate = test_df_sorted["true_positive"].mean()

print(f"Precision in Top {budget_fraction*100:.0f}%: {precision_top_k:.4f}")
print(f"Overall Positive Rate: {overall_positive_rate:.4f}")
print(f"Lift: {precision_top_k / overall_positive_rate:.2f}x improvement")

# Plot precision lift
plt.figure(figsize=(6, 4))
bars = plt.bar(["Top 20% Precision", "Overall Rate"], [precision_top_k, overall_positive_rate], color=["blue", "gray"])
plt.title("Targeting Precision vs. Random")
plt.ylabel("Precision")
plt.ylim(0, 1)
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()


In [ ]:

# 🔍 Detailed SHAP Value Extraction and Plotting for Each Model

# Preprocessed feature names
feature_names = preprocessor.get_feature_names_out()

# SHAP for Propensity Model (Logistic Regression)
explainer_prop = shap.Explainer(dr._model_propensity.predict_proba, X_train_proc.toarray() if hasattr(X_train_proc, 'toarray') else X_train_proc)
shap_values_prop = explainer_prop(X_test_proc.toarray() if hasattr(X_test_proc, 'toarray') else X_test_proc)

print("SHAP Summary for Propensity Model:")
shap.plots.beeswarm(shap_values_prop, max_display=10, show=False)
plt.title("SHAP Summary - Propensity Model")
plt.show()

# SHAP for Outcome Model (Gradient Boosting)
explainer_reg = shap.Explainer(dr._model_regression.predict, X_train_proc)
shap_values_reg = explainer_reg(X_test_proc)

print("SHAP Summary for Outcome Model:")
shap.plots.beeswarm(shap_values_reg, max_display=10, show=False)
plt.title("SHAP Summary - Outcome Model")
plt.show()

# SHAP for Final CATE Model (Neural Network)
final_model_internal = dr.model_final.model_
explainer_final = shap.Explainer(final_model_internal, X_test_proc)
shap_values_final = explainer_final(X_test_proc)

print("SHAP Summary for Final CATE Model:")
shap.plots.beeswarm(shap_values_final, max_display=10, show=False)
plt.title("SHAP Summary - Final CATE Model")
plt.show()
